In [0]:
# Creating month wise partitioned parquet file from csv
from pyspark.sql.functions import *
from pyspark.sql.types import *

schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_date", DateType(), True),
    StructField("order_time", StringType(), True)])

orders = spark.read.format("csv").option("header",True).schema(schema).load("abfss://datalake@enterprisedatalake12.dfs.core.windows.net/bronze/orders.csv")
orders_concat =orders.withColumn("Order_Hour",substring(orders.order_time,0,2).cast("Integer")).withColumn("Order_Month", substring(orders.order_date,6,2).cast("Integer"))
orders_concat.write.format('parquet').partitionBy("Order_Month").mode("overwrite").save("abfss://datalake@enterprisedatalake12.dfs.core.windows.net/silver/orders")



In [0]:
# df = spark.read.format("parquet").load("abfss://raw@enterprisedatalake12.dfs.core.windows.net/silver/orders")
# df.display()

In [0]:
# df = spark.read.format("csv").load("abfss://raw@enterprisedatalake12.dfs.core.windows.net/silver/orders")
# df.display()

In [0]:
# Create Permanent Delta table from CSV
location = "abfss://datalake@enterprisedatalake12.dfs.core.windows.net/bronze/order_details.csv"
file_type = "csv"

infer_schema = "True"
first_row_is_header = "True"
delimiter = ","

delta_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(location)


permanent_table_name = "order_details_delta"
if spark.catalog.tableExists(permanent_table_name):
  delta_df.write.format("delta").mode("overwrite").saveAsTable(permanent_table_name)
else:
  pass